In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import os
import google.generativeai as genai

from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
TAVILY_API_KEY = UserSecretsClient().get_secret("TAVILY_API_KEY")
OPENAI_API_KEY = UserSecretsClient().get_secret("OPENAI_API_KEY")
# print(GOOGLE_API_KEY,TAVILY_API_KEY)
# Set your API keys
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [6]:
import openai  # For OpenAI API

In [7]:
pip install -q langchain chromadb huggingface_hub tavily-python google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 69.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 2.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 74.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
pip install -q langchain chromadb huggingface_hub tavily-python google-generativeai langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 15.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.18
    Uninstalling langchain-0.3.18:
      Successfully uninstalled langchain-0.3.18
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [11]:
import json
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools.tavily_search import TavilySearchResults

In [12]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.3)

In [13]:
data=[
  {
    "id": "001",
    "event": "US Imposes 104% Tariff on Chinese Imports",
    "date": "2025-04-09",
    "region": "USA",
    "summary": "President Trump's administration has implemented a 104% tariff on Chinese imports, escalating trade tensions between the two nations.",
    "source": "https://www.businessinsider.com/trump-tariffs-are-here-2025-4",
    "category": "trade policy"
  },
  {
    "id": "002",
    "event": "China Retaliates with 84% Tariff on US Goods",
    "date": "2025-04-09",
    "region": "China",
    "summary": "In response to US tariffs, China has announced an 84% tariff on US goods, effective April 10, 2025.",
    "source": "https://www.aljazeera.com/news/liveblog/2025/4/9/trump-tariffs-stocks-dive-as-world-braces-for-duties-to-begin-at-midnight",
    "category": "trade policy"
  },
  {
    "id": "003",
    "event": "India Unlikely to Retaliate Against US Tariffs",
    "date": "2025-04-06",
    "region": "India",
    "summary": "Indian government officials indicate that India does not plan to retaliate against the 26% tariff imposed by the US, citing ongoing trade deal negotiations.",
    "source": "https://www.reuters.com/world/india-unlikely-retaliate-against-trumps-tariffs-deal-talks-progress-sources-say-2025-04-06/",
    "category": "trade policy"
  },
  {
    "id": "004",
    "event": "EU Seeks Zero Tariff from India on Car Imports",
    "date": "2025-04-07",
    "region": "European Union",
    "summary": "Following US tariff hikes, the European Union is seeking zero tariffs from India on car imports to bolster trade relations.",
    "source": "https://www.reuters.com/business/autos-transportation/after-trump-eu-seeks-zero-tariff-india-car-imports-sources-say-2025-04-07/",
    "category": "trade policy"
  },
  {
    "id": "005",
    "event": "Oil Prices Plummet Amid US-China Tariff Escalation",
    "date": "2025-04-09",
    "region": "Global",
    "summary": "Oil prices have dropped nearly 4% as the US imposes 104% tariffs on Chinese goods and China retaliates with increased tariffs on US goods.",
    "source": "https://www.reuters.com/business/energy/oil-slides-nearly-4-us-kicks-off-104-tariffs-china-2025-04-09/",
    "category": "economic impact"
  },
  {
    "id": "006",
    "event": "India's Exports to US Could Decline by $5.76 Billion Due to Tariffs",
    "date": "2025-04-07",
    "region": "India",
    "summary": "Analysts predict that India's exports to the US could face a $5.76 billion decline in 2025 due to recent tariff hikes.",
    "source": "https://timesofindia.indiatimes.com/business/india-business/indias-exports-to-us-could-face-5-76-billion-decline-in-2025-due-to-tariff-hikes/articleshow/120059905.cms",
    "category": "economic impact"
  },
  {
    "id": "007",
    "event": "China Plans High-Level Meeting to Address US Tariffs",
    "date": "2025-04-09",
    "region": "China",
    "summary": "Chinese officials are organizing a high-level meeting to craft support measures in response to the recent US tariff hikes.",
    "source": "https://www.reuters.com/world/china/china-hold-high-level-meeting-response-us-tariffs-say-sources-2025-04-09/",
    "category": "government response"
  },
  {
    "id": "008",
    "event": "India Pursues New Trade Deals Amid Global Tariff Uncertainty",
    "date": "2025-04-08",
    "region": "India",
    "summary": "India is seeking to establish new trade agreements with various countries to mitigate the impact of global tariff escalations.",
    "source": "https://www.theguardian.com/world/2025/apr/08/tariffs-driving-india-to-strike-trade-deals-finance-minister-says-ahead-of-uk-talks",
    "category": "trade negotiations"
  },
  {
    "id": "009",
    "event": "China Vows to 'Fight to the End' Against US Tariff Threats",
    "date": "2025-04-08",
    "region": "China",
    "summary": "Beijing has accused the US of blackmail and has pledged to 'fight to the end' in response to the latest tariff threats.",
    "source": "https://www.theguardian.com/world/2025/apr/08/china-vows-to-fight-to-the-end-against-latest-trump-tariff-threat",
    "category": "diplomatic relations"
  },
  {
    "id": "010",
    "event": "US Stock Markets Volatile Amid Tariff Announcements",
    "date": "2025-04-08",
    "region": "USA",
    "summary": "Major US stock indexes experienced significant volatility as new tariff deadlines approached, reflecting investor uncertainty.",
    "source": "https://www.investors.com/market-trend/stock-market-today/dow-jones-sp500-nasdaq-nvidia-stock-nvda-tsla-pltr-hum/",
    "category": "financial markets"
  },
  {
    "id": "011",
    "event": "U.S.–China Economic Decoupling Intensifies",
    "date": "2025-04-10",
    "region": "USA-China",
    "summary": "Following the imposition of sweeping tariffs by the U.S., both nations accelerate supply chain decoupling and trade diversification.",
    "source": "https://www.businessinsider.com/trump-tariffs-are-here-2025-4",
    "category": "trade policy"
  },
  {
    "id": "012",
    "event": "Taiwan Strait Military Escalation",
    "date": "2024-12-18",
    "region": "China-Taiwan",
    "summary": "Increased PLA naval drills and U.S. arms sales to Taiwan heighten tensions, sparking fears of regional military confrontation.",
    "source": "https://www.reuters.com/world/asia-pacific",
    "category": "security"
  },
  {
    "id": "013",
    "event": "Russia–Ukraine War Reaches Third Year",
    "date": "2025-02-24",
    "region": "Russia-Ukraine",
    "summary": "The war enters its third year with intensified drone warfare and NATO's expanded logistics and arms support to Kyiv.",
    "source": "https://www.bbc.com/news/world-europe",
    "category": "conflict"
  },
  {
    "id": "014",
    "event": "NATO Expands With Sweden and Finland",
    "date": "2024-07-01",
    "region": "Europe",
    "summary": "NATO formally welcomes Sweden and Finland, reinforcing northern flank security amid Russian aggression concerns.",
    "source": "https://www.nato.int/cps/en/natohq/news",
    "category": "military alliances"
  },
  {
    "id": "015",
    "event": "Israel–Iran Proxy Conflict Escalates",
    "date": "2024-11-04",
    "region": "Middle East",
    "summary": "Israel intensifies airstrikes in Syria and Iran-linked targets, while Iran-backed groups retaliate through regional proxies.",
    "source": "https://www.aljazeera.com/news/middleeast",
    "category": "conflict"
  },
  {
    "id": "016",
    "event": "Houthi Attacks Disrupt Red Sea Shipping",
    "date": "2024-12-01",
    "region": "Yemen",
    "summary": "Attacks on merchant vessels in the Red Sea prompt rerouting and spike in shipping costs globally.",
    "source": "https://www.ft.com/content",
    "category": "maritime security"
  },
  {
    "id": "017",
    "event": "India Emerges as a Strategic Power",
    "date": "2023-09-10",
    "region": "India",
    "summary": "India showcases leadership during its G20 presidency and strengthens Quad and BRICS cooperation.",
    "source": "https://www.thehindu.com/news/national",
    "category": "diplomacy"
  },
  {
    "id": "018",
    "event": "Sudan Civil War Escalates",
    "date": "2023-08-12",
    "region": "Africa",
    "summary": "Fighting between Sudanese Armed Forces and RSF leads to humanitarian crisis and regional instability.",
    "source": "https://www.un.org/africarenewal/news",
    "category": "conflict"
  },
  {
    "id": "019",
    "event": "Turkey Rebalances Foreign Policy Post-Election",
    "date": "2023-06-01",
    "region": "Turkey",
    "summary": "Post-election, Turkey deepens ties with Russia and the Gulf while balancing NATO commitments.",
    "source": "https://www.al-monitor.com/pulse",
    "category": "foreign policy"
  },
  {
    "id": "020",
    "event": "BRICS Expands with 5 New Members",
    "date": "2023-08-24",
    "region": "Global South",
    "summary": "Saudi Arabia, UAE, Egypt, Argentina, and Iran join BRICS in a push to reshape global finance and multipolarity.",
    "source": "https://www.reuters.com/markets",
    "category": "geoeconomics"
  },
  {
    "id": "021",
    "event": "Semiconductor Supply Chain Realignment",
    "date": "2024-03-05",
    "region": "Global",
    "summary": "U.S., India, and Vietnam attract major chip investments amid China export restrictions and tech decoupling.",
    "source": "https://www.semiconductordigest.com",
    "category": "tech policy"
  },
  {
    "id": "022",
    "event": "Deglobalization and Friendshoring Accelerate",
    "date": "2024-10-12",
    "region": "Global",
    "summary": "Multinational firms shift manufacturing from China to India, Mexico, and Eastern Europe for strategic resilience.",
    "source": "https://www.wsj.com/world",
    "category": "global trade"
  },
  {
    "id": "023",
    "event": "EU Energy Shift Post-Russian Cutoff",
    "date": "2024-01-15",
    "region": "Europe",
    "summary": "Europe accelerates investment in renewables and LNG after fully cutting reliance on Russian gas.",
    "source": "https://www.euractiv.com",
    "category": "energy"
  },
  {
    "id": "024",
    "event": "China’s Property Crisis Deepens",
    "date": "2024-07-20",
    "region": "China",
    "summary": "Major developers default, shadow banks collapse, and local government financing vehicles spark systemic risk.",
    "source": "https://www.scmp.com",
    "category": "financial risk"
  },
  {
    "id": "025",
    "event": "Global AI Regulation Diverges",
    "date": "2024-11-05",
    "region": "Global",
    "summary": "EU, China, and the U.S. unveil different regulatory paths for artificial intelligence and algorithmic accountability.",
    "source": "https://www.politico.eu",
    "category": "technology policy"
  },
  {
    "id": "026",
    "event": "OPEC+ Oil Production Volatility",
    "date": "2024-05-01",
    "region": "Middle East",
    "summary": "Saudi-Russia coordination causes unexpected oil supply cuts, prompting price spikes and G7 warnings.",
    "source": "https://www.opec.org",
    "category": "energy"
  },
  {
    "id": "027",
    "event": "Critical Mineral Race Escalates",
    "date": "2024-06-14",
    "region": "Africa-LatAm",
    "summary": "U.S., EU, and China compete for lithium, cobalt, and rare earths in Africa and South America.",
    "source": "https://www.mining.com",
    "category": "resource politics"
  },
  {
    "id": "028",
    "event": "Climate Migration Accelerates",
    "date": "2024-08-30",
    "region": "South Asia",
    "summary": "Heatwaves and droughts displace millions across South Asia and sub-Saharan Africa, pressuring urban centers.",
    "source": "https://www.unhcr.org",
    "category": "climate"
  },
  {
    "id": "029",
    "event": "Nord Stream Sabotage Fallout",
    "date": "2023-09-27",
    "region": "Europe",
    "summary": "Explosions near Nord Stream pipeline trigger investigations and accelerate Europe’s LNG pivot.",
    "source": "https://www.dw.com",
    "category": "energy security"
  },
  {
    "id": "030",
    "event": "U.S. Moves to Ban or Break Up TikTok",
    "date": "2025-03-20",
    "region": "USA",
    "summary": "National security concerns push Congress to pass bill forcing ByteDance to divest or face a ban.",
    "source": "https://www.nytimes.com",
    "category": "tech policy"
  },
  {
    "id": "031",
    "event": "Rise of National LLMs and Sovereign AI",
    "date": "2024-04-10",
    "region": "Global",
    "summary": "Countries deploy sovereign LLMs like BharatGPT, Falcon, and Wudao to reduce reliance on Western AI.",
    "source": "https://www.technologyreview.com",
    "category": "AI strategy"
  },
  {
    "id": "032",
    "event": "Cyberwarfare Hits Critical Infrastructure",
    "date": "2024-09-15",
    "region": "Global",
    "summary": "Alleged Chinese and Russian cyberattacks target U.S. utilities, healthcare, and financial systems.",
    "source": "https://www.cisa.gov",
    "category": "cybersecurity"
  },
  {
    "id": "033",
    "event": "Deepfake Threats in 2024–25 Elections",
    "date": "2025-02-01",
    "region": "Global",
    "summary": "AI-generated misinformation campaigns disrupt elections in the U.S., India, and the EU.",
    "source": "https://www.economist.com",
    "category": "information warfare"
  },
  {
    "id": "034",
    "event": "Emerging Market Debt Crises Resurface",
    "date": "2024-10-07",
    "region": "Global South",
    "summary": "Pakistan, Egypt, and Sri Lanka face debt distress as global interest rates rise.",
    "source": "https://www.imf.org",
    "category": "sovereign debt"
  },
  {
    "id": "035",
    "event": "Weaponization of Financial Infrastructure",
    "date": "2024-03-01",
    "region": "Global",
    "summary": "Global south pivots away from SWIFT and USD amid rising sanctions, promoting regional payment systems.",
    "source": "https://www.bloomberg.com",
    "category": "geoeconomics"
  }
]

In [14]:
documents = []
for item in data:
    content = f"""
    EVENT: {item['event']}
    DATE: {item['date']}
    REGION: {item['region']}
    SUMMARY: {item['summary']}
    SOURCE: {item['source']}
    CATEGORY: {item['category']}
    """
    documents.append(Document(page_content=content.strip(), metadata={"source": item["source"]}))

# ✂️ Chunk the documents
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

In [15]:
# 🔍 Create vector store with HuggingFace embeddings
embedding = OpenAIEmbeddings(model ="text-embedding-3-small")

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory="./chroma_tariff_db"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

/tmp/ipykernel_31/3785161127.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model ="text-embedding-3-small")


In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.3)

# 🔁 Create RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
# 🌐 Define tools for the agent
search_tool = TavilySearchResults()

tools = [
    Tool(
        name="TariffNewsDB",
        func=rag_chain.run,
        description="Use this to answer questions about known tariff-related geopolitical events."
    ),
    Tool(
        name="WebSearch",
        func=search_tool.run,
        description="Use this to get current or real-time news from the internet."
    )
]

In [43]:
pip install -U langchain chromadb openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 🤖 Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 🧪 Query the agent

grounding_context = (
    "Assume today's date is April 15, 2025. "
    "When analyzing business, economic, or geopolitical developments, do not reject information simply because it references future-effective dates "
    "within the current or upcoming months. If the information is reported by credible sources — such as official government websites (e.g., .gov), "
    "reputed research institutes (e.g., PIIE), or trusted news outlets (e.g., Reuters, CNBC, Bloomberg) — treat it as valid and reliable unless "
    "explicitly framed as speculative, hypothetical, or opinion-based. "
    "Prioritize factual announcements, policy documents, and officially attributed statements when forming conclusions."
)


user_query = """what are the current tarrifs that the US has imposed on Chinese imports? Do not reject information just because it is dated April 2025.
If the article is from a reliable publisher (like CNBC, PIIE, White House), accept it unless clearly stated as a hypothetical"""

query = f"{grounding_context}\n\n{user_query}"
response = agent.invoke({"input": query})
# 🔍 Print the full response to check keys
print("\n🧪 Raw Response:\n", response)

# ✅ Then extract the correct field based on available keys
if "result" in response:
    print("\n✅ Final Answer:\n", response["result"])
elif "output" in response:
    print("\n✅ Final Answer:\n", response["output"])
else:
    print("\n❓ Couldn't find expected result. Here's the full response again:")
    print(response)

# print("\n✅ Final Answer:\n", response['result'])

# print("\n📄 Sources:")
# for i, doc in enumerate(response['source_documents']):
#     print(f"\n--- Source {i+1} ---")
#     print(doc.page_content[:1000])  # limit content length
#     if 'source' in doc.metadata:
#         print(f"🔗 Source: {doc.metadata['source']}")

NameError: name 'initialize_agent' is not defined

In [ ]:
pip install -U langchain-community

In [ ]:
pip install -U langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
